# PROBLEMA DI STOKES

# Import di tutti i pacchetti

In [52]:
nome = 'nome' #Nome che voglio dare alla simulazione così da distinguere loss e plot per tutto

import torch

import sys
sys.path.append('C:/Users/Andrea/Desktop/Poli/Tesi magistrale/reporitory_SISSA_PoliTO')

from pina.problem import SpatialProblem, ParametricProblem
from pina.operators import laplacian, grad, div
from pina import Condition, LabelTensor
from pina.geometry import CartesianDomain
from pina.equation import SystemEquation, Equation
from pina.model import MultiFeedForward

import argparse
from torch.nn import Softplus

from pina import Plotter, Trainer
from pina.model import FeedForward
from pina.solvers import PINN

# Definizione del problema
Questo problema ha un unico parametro
- $\mu\in[0.5,1.5]$

In [53]:
"""Stokes Problem """
# ===================================================== #
#             The Stokes class is defined               #
#       inheriting from SpatialProblem. We  denote:     #
#           ux --> field variable velocity along x      #
#           uy --> field variable velocity along y      #
#           p --> field variable pressure               #
#           x,y --> spatial variables                   #
#                                                       #
#           https://arx#iv.org/pdf/2110.13530.pdf       #
# ===================================================== #
alfa = 0.008

class Stokes(SpatialProblem, ParametricProblem):

    # assign output/ spatial variables
    output_variables = ['vx', 'vy', 'p', 'ux', 'uy', 'r', 'zx', 'zy'] 
    #vx, vy is the 2-dim state variable, #ux, uy is the 2-dim control variable. 
    xmin = 0
    xmax = 1
    ymin = 0
    ymax = 2
    mumin = 0.5
    mumax = 1.5

    xrange = [xmin, xmax]
    yrange = [ymin, ymax]
    murange = [mumin, mumax]
    
    #r è la variabile aggiunta della pressione, z è la aggiunta del campo di velocità
    spatial_domain = CartesianDomain({'x': xrange, 'y': yrange})
    parameter_domain = CartesianDomain({'mu': murange})




    
    #Prima ci sono tutte le equazioni sulle variabili aggiunte, poi tutte quelle per le variabili non aggiunte
    # PDE
    def momentum_ad_x(input_, output_):
        delta = laplacian(output_, input_, components = ['zx'], d = ['x'])
        return -0.1 * delta + grad(output_, input_, components = ['r'], d = ['x']) - input_.extract(['y']) + output_.extract(['vx']) 
            
    def momentum_ad_y(input_, output_):
        delta = laplacian(output_, input_, components = ['zy'], d = ['y'])
        return -0.1 * delta + grad(output_, input_, components = ['r'], d = ['y']) 
    
    def continuity_ad(input_, output_):
        return grad(output_, input_, components = ['zx'], d = ['x']) + grad(output_, input_, components = ['zy'], d = ['y'])

    # BOUNDARY CONDITIONS on adjuncted variables
    # Dirichlet
    def dirichlet1_ad(input_, output_):
        return output_.extract(['zx'])

    def dirichlet2_ad(input_, output_):
        return output_.extract(['zy'])

    # Neumann
    def neumann1_ad(input_, output_):
        return -output_.extract(['r']) + 0.1*grad(output_, input_, components = ['zx'], d = ['x'])
    def neumann2_ad(input_, output_):
        return output_.extract(['zy'])

    ############################################################################

    # Momentum Equations
    def momentum_x(input_, output_):
        delta = laplacian(output_, input_, components = ['vx'], d = ['x'])
        return -0.1 * delta + grad(output_, input_, components = ['p'], d = ['x'])  - output_.extract(['ux'])
 
    def momentum_y(input_, output_):
        delta = laplacian(output_, input_, components = ['vy'], d = ['y'])
        return -0.1 * delta + grad(output_, input_, components = ['p'], d = ['y']) + input_.extract(['mu'])  - output_.extract(['uy'])

    # Continuity equation
    def continuity(input_, output_):
        return grad(output_, input_, components = ['vx'], d = ['x']) + grad(output_, input_, components = ['vy'], d = ['y'])

    # BOUNDARY CONDITIONS on principal variable
    # Dirichlet
    def dirichlet1(input_, output_):
        return output_.extract(['vx']) - input_.extract(['y'])

    def dirichlet2(input_, output_):
        return output_.extract(['vy'])

    # Neumann
    def neumann1(input_, output_):
        return -output_.extract(['p']) + 0.1*grad(output_, input_, components = ['vx'], d = ['x'])

    def neumann2(input_, output_):
        return output_.extract(['vy'])


    
    
    #Problem Statement
    conditions = {
        'gamma_above': Condition(location=CartesianDomain({'x': xrange, 'y':  2, 'mu': murange}), equation=SystemEquation([dirichlet1, dirichlet2, dirichlet1_ad, dirichlet2_ad])), #Dirichlet
        'gamma_left': Condition(location=CartesianDomain({'x': 0, 'y': yrange, 'mu': murange}), equation=SystemEquation([dirichlet1, dirichlet2, dirichlet1_ad, dirichlet2_ad])), #Dirichlet
        'gamma_below': Condition(location=CartesianDomain({'x':  xrange, 'y': 0, 'mu': murange}), equation=SystemEquation([dirichlet1, dirichlet2, dirichlet1_ad, dirichlet2_ad])), #Dirichlet
        'gamma_right':  Condition(location=CartesianDomain({'x': 1, 'y': yrange, 'mu': murange}), equation=SystemEquation([neumann1, neumann2, neumann1_ad, neumann2_ad])), #Neumann
        'D': Condition(location=CartesianDomain({'x': xrange, 'y': yrange, 'mu': murange}), equation=SystemEquation([momentum_x,momentum_y,continuity, 
                                                                                                                     momentum_ad_x,momentum_ad_y,continuity_ad]))
    }


# DEFINIZIONE ARCHITETTURA DELLA PINN

In [54]:
class CustomMultiDFF(MultiFeedForward):

    def __init__(self, dff_dict):
        super().__init__(dff_dict)

    #Questo si usa al posto della equazione che "manca", altrimenti viene male
    def forward(self, x):
        out = self.uu(x)
        out.labels = ['vx', 'vy', 'p', 'ux', 'uy', 'r']
        z = LabelTensor(alfa*out.extract(['ux', 'uy']), ['zx', 'zy'])
        
        return out.append(z)

In [55]:
if __name__ == "__main__":
    
    seed = 316680
    torch.manual_seed(seed)
    
    epochs = 10

    parser = argparse.ArgumentParser(description="Run PINA")
    parser.add_argument("--load", help = "directory to save or load file", type = str)
    parser.add_argument("--epochs", help = "extra features", type = int, default = epochs)
    parser.add_argument('-f') #Serve per risolvere l'errore di sotto
    args = parser.parse_args()
    
    # create problem and discretise domain
    stokes_opc = Stokes()
    stokes_opc.discretise_domain(n = 1800, mode = 'lh', variables = ['x', 'y'], locations = ['gamma_above', 'gamma_left', 'gamma_below', 'gamma_right'])
    stokes_opc.discretise_domain(n = 400, mode = 'lh', variables = ['x', 'y'], locations = ['D'])
    stokes_opc.discretise_domain(n = 10, mode = 'lh', variables = ['mu'], locations = ['gamma_above', 'gamma_left', 'gamma_below', 'gamma_right'])
    stokes_opc.discretise_domain(n = 10, mode = 'lh', variables = ['mu'], locations = ['D'])
    
    # make the model
    model = CustomMultiDFF(
        {'uu': {
                'input_dimensions': 3,
                'output_dimensions': 6,
                'layers': [40, 40, 40, 40],
                'func': Softplus, },})
    
    # make the pinn
    pinn = PINN(problem = stokes_opc, model = model, optimizer_kwargs={'lr' : 0.003})
    
    # create trainer
    directory = 'pina.navier_stokes'
    trainer = Trainer(solver=pinn, accelerator='cpu', max_epochs=args.epochs, default_root_dir=directory)

    #Training
    trainer.train()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name        | Type    | Params
----------------------------------------
0 | _loss       | MSELoss | 0     
1 | _neural_net | Network | 5.3 K 
----------------------------------------
5.3 K     Trainable params
0         Non-trainable params
5.3 K     Total params
0.021     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 1/1 [00:00<00:00,  1.69it/s, v_num=17, gamma_above_loss=0.0721, gamma_left_loss=0.0209, gamma_below_loss=0.0513, gamma_right_loss=0.00301, D_loss=0.0132, mean_loss=0.0321]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 1/1 [00:00<00:00,  1.66it/s, v_num=17, gamma_above_loss=0.0721, gamma_left_loss=0.0209, gamma_below_loss=0.0513, gamma_right_loss=0.00301, D_loss=0.0132, mean_loss=0.0321]


# GRAFICI

In [ ]:
plotter = Plotter()
plotter.plot(pinn, fixed_variables={'mu': 0}, components='vx')
plt.gcf().savefig(nome + '_vx.pdf', format='pdf')

In [ ]:
plotter = Plotter()
plotter.plot(pinn, fixed_variables={'mu': 0}, components='vy')
plt.gcf().savefig(nome + '_vy.pdf', format='pdf')

In [ ]:
plotter = Plotter()
plotter.plot(pinn, fixed_variables={'mu': 0}, components='ux')
plt.gcf().savefig(nome + '_ux.pdf', format='pdf')

In [ ]:
plotter = Plotter()
plotter.plot(pinn, fixed_variables={'mu': 0}, components='uy')
plt.gcf().savefig(nome + '_uy.pdf', format='pdf')

In [ ]:
plotter = Plotter()
plotter.plot(pinn, fixed_variables={'mu': 0}, components='p')
plt.gcf().savefig(nome + '_p.pdf', format='pdf')

In [ ]:
plotter = Plotter()
plotter.plot(pinn, fixed_variables={'mu': 0}, components='r')
plt.gcf().savefig(nome + '_r.pdf', format='pdf')

In [ ]:
plotter = Plotter()
plotter.plot(pinn, fixed_variables={'mu': 0}, components='zx')
plt.gcf().savefig(nome + '_zx.pdf', format='pdf')

In [ ]:
plotter = Plotter()
plotter.plot(pinn, fixed_variables={'mu': 0}, components='zy')
plt.gcf().savefig(nome + '_zy.pdf', format='pdf')

# Loss

In [ ]:
#Qui salvo la loss function
andamento_loss = trainer._model.lossVec
def salva_variabile(file, variabile):
    with open(file, 'w') as f:
        f.write(repr(variabile))

# Chiama la funzione per salvare la variabile
salva_variabile('loss_'+ nome +'.txt', andamento_loss)

# Grafico loss
plt.loglog(andamento_loss)
plt.gcf().savefig(nome + 'grafico_loss.pdf', format='pdf') # Qui per salvare il grafico della loss